# ABM

## Imports / Data Conversion

In [3]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
# import panel as pn
# import panel.widgets as pnw
import random
from tqdm import tqdm, trange
from time import sleep

data_path = '../' #set to wherever the data files are, will be used on every input

In [3]:
def kmtoNaut(km):
    return km / 1.852

In [10]:
ports = pd.read_csv((data_path +'ports.csv'))
data = pd.read_csv((data_path + 'clean_distances.csv'))
origin = pd.read_csv((data_path + 'origin_ports.csv'))

In [14]:
distances = data[["prev_port", "next_port", "distance"]]

In [18]:
distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes

prev_port      int64
next_port      int64
distance     float64
dtype: object

## Playground 

In [19]:
G = nx.MultiGraph()
G.add_nodes_from(N) #instatiate the ports as nodes of network DOES THIS WORK
for i in range(len(distances)): #create bi-directional edges with an attribute for length 
       G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
grid = NetworkGrid(G) #Define Mesa Grid as the just created Network to allow for shipping only in routes

In [263]:
e = 0
for index, row in origin.iterrows():
    if pd.isna(ports.iloc[index]["INDEX_NO"]):
        e += 1

e

49

In [88]:
ports[ports["Unnamed: 0"]==286]
# ports.head()

,Unnamed: 0,PORT_NAME,INDEX_NO,coords
29,286,Hannan Port,NaN,"((135.31110022775667, 34.421924371272326),)"


In [75]:
N = data["next_port"].tolist()
N = list(set(N))
len(N)

2842

In [73]:
np.random.choice(origin["Ref"], size=None, replace=True,  p=origin["PROB"])

2560.0

In [81]:
G.edges(12447)


MultiEdgeDataView([(12447, 286.0)])

In [236]:
random.sample(G.nodes, k=1)[0]

3579

In [ ]:
class ShippingNetwork(Model):
    def __init__(self, distances, major_ports, S=2):
        self.major_ports = major_ports
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True
        self.G = nx.from_pandas_edgelist(distances, "prev_port", "next_port",edge_attr= "distance")
         #instatiate the ports as nodes of network     
        
        self.grid = NetworkGrid(self.G) #Define Mesa Grid as the just created Network to allow for shipping only in routes
       

        #create ability to remove edges mid-model
        def network_change(self, change_type, change_edge):
            if change_type == "add":
                self.G.add_edge(self.change_edge[0], self.change_edge[1], length=self.change_edge[2])
            if change_type == "remove":
                self.G.remove_edge(self.change_edge[0],self.change_edge[1]) #can we identify an edge by node 1, node 2 & distance?
            #update model with new grid    
            return NetworkGrid(self.G), G


        #create agents 
        Ships = []
       
        for i in tqdm(range(self.num_ships), desc="Placing Ships"):
        
            a = Ship(i+1, self, self.G, self.major_ports)
            self.schedule.add(a)
            #append to list of ships
            Ships.append(a)
        
            #place agent on origin node
            self.grid.place_agent(a, a.start)



        self.datacollector = DataCollector(
            model_reporters={"Graph":"G"},
            agent_reporters={"Type": "ship_class", "Position": "position", "Destination":"destination", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "Route":"current_route", "Route Changes":"route_chng" })


    def step(self, change_type='', change_edge=[]):
        #check network for changes
        if change_type != '' :
            self.grid, self.G = network_change(change_type, change_edge)

        self.schedule.step()     #Run each Agents
        self.datacollector.collect(self)

## Model

In [59]:
class ShippingNetwork(Model):
    def __init__(self, N, distances, major_ports, S=2):
        N
        self.major_ports = major_ports
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True

        self.G = nx.MultiGraph()
        self.G.add_nodes_from(N) #instatiate the ports as nodes of network
        
       
        for i in tqdm(range(len(distances)), desc="Building Network" ): #create bi-directional edges with an attribute for length 
            self.G.add_edge(distances.iloc[i][0], distances.iloc[i][1], length=distances.iloc[i][2])
            
        
        self.grid = NetworkGrid(self.G) #Define Mesa Grid as the just created Network to allow for shipping only in routes
       

        #create ability to remove edges mid-model
        def network_change(self, change_type, change_edge):
            if change_type == "add":
                self.G.add_edge(self.change_edge[0], self.change_edge[1], length=self.change_edge[2])
            if change_type == "remove":
                self.G.remove_edge(self.change_edge[0],self.change_edge[1]) #can we identify an edge by node 1, node 2 & distance?
            #update model with new grid    
            return NetworkGrid(self.G), G


        #create agents 
        Ships = []
       
        for i in tqdm(range(self.num_ships), desc="Placing Ships"):
        
            a = Ship(i+1, self, self.G,  major_ports)
            self.schedule.add(a)
            #append to list of ships
            Ships.append(a)
        
            #place agent on origin node
            self.grid.place_agent(a, start_port)



        self.datacollector = DataCollector(
            model_reporters={"Graph":"G"},
            agent_reporters={"Position": "position", "Destination":"destination", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "Route":"current_route", "Route Changes":"route_chng" })


    def step(self, change_type='', change_edge=[]):
        #check network for changes
        if change_type != '' :
            self.grid, self.G = network_change(change_type, change_edge)

        self.schedule.step()     #Run each Agents
        self.datacollector.collect(self)

In [53]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, major_ports, s=13.0, f = 20):
        super().__init__(unique_id, model)
        
        self.major_ports = major_ports
        self.G = G
        self.ship_class = np.random.choice(["Ultralarge","Normal", "Small"], 1, p=[0.5, 0.25, 0.25])
        self.start_port = self.origin()
        self.start = self.start_port[0]
        self.destination = self.dest() #sample a destination
        #We sample the origin port from a list of the 50 biggest ports world, with the prob = TAU of the port / TAU of all origin ports for 2017
        self.ports =  [*self.start_port, *self.destination]
        
        self.foresight = np.random.poisson(f)
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = s*24*1.852 #speed is given in knots, with 1 knot being 1 nautical mile per hour. Since the model works with distances in km, we convert here (1 nm = 1.852m)
        self.not_reachable = 0 #counter for needing to abandon a route 
        self.new_dest_failed = 0 #counter for ships not able to reach any of the ports
        self.init_route, self.init_dist  = self.routing() #We keep a copy of the entire itinerary / distance traveled
        
        self.current_route, self.current_dist = self.init_route.copy(), self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.position = self.current_route[0]
        self.next_position = self.current_route[1]
  
        self.itinerary = [self.position]
        self.distance_traveled = 0
        self.unique_id = unique_id
        
        self.step_size = self.ident_distance()
        self.route_chng = 0
        self.routes = 0
        
        

    def origin(self):
        """
        Sample origin based on ship type.
        """ 
        
        if self.ship_class == "Ultralarge":
            start_port = np.random.choice(self.major_ports["Ref"],  p=self.major_ports["PROB"])

        elif self.ship_class == "Normal":
            start_port = np.random.choice(self.major_ports["Ref"],  p=self.major_ports["PROB"])

        else:
            start_port = float(random.sample(self.G.nodes, k=1)[0])
        
        return [start_port]
        

    def dest(self):
        """
        Sample destinations
        """
        if self.ship_class == "Ultralarge":
            k = np.random.randint(1, high = 6)
            end = np.random.choice(self.major_ports["Ref"], size=k,  p=self.major_ports["PROB"]).tolist()
            
            
        elif self.ship_class == "Normal":
            k = np.random.randint(1, high =11)
            end = np.random.choice(self.major_ports["Ref"], size=k,  p=self.major_ports["PROB"]).tolist()

        else:
            k = np.random.randint(1, high =6)
            end = [float(i) for i in random.sample(self.G.nodes, k=k)]

        return end

    def routing(self):
        """
        A greedy version of Travelling Salesman algorithm.
        Takes in a list of ports, with the first port being the origin.
        It loops to find the closest port. Returns a list of ports to visit (an itinerary) and the overall distance.
        """
        ports = self.ports.copy()
        overall_distance = list()
        itinerary = list() 
        not_reached = 0
        
        try:
            for j in range(len(ports)):
                distance = dict()
                try:
                    for i in range(1,len(ports)): #look for the closest port
                        distance[ports[i]] = nx.shortest_path_length(self.G, ports[0] , ports[i], weight='distance')
                    next_stop = min(distance, key=distance.get)
                    itinerary.append(nx.shortest_path(self.G, ports[0], next_stop, weight = 'distance')) #add the route to the closest port to the itinerary
                    overall_distance.append(distance.get(next_stop)) #add distance to the closest port
                    ports.pop(0)
                    ind = ports.index(next_stop)
                    ports.pop(ind)
                    ports.insert(0, next_stop)

                except nx.NetworkXNoPath:
                    self.not_reachable += 1 #global counter
                    not_reached += 1 #loccal counter

        except ValueError: #handle list end
            pass

        if not_reached == len(ports): #if no routes possible routes found, reassign destination
            self.new_dest_failed =+1
            self.destination = self.dest()
            if self.new_dest_failed >1 :
                self.start_port = self.origin()
                self.new_dest_failed = 0 # clear the counter
                return self.routing()
             
            return self.routing() #recursion brrrr
            
        assert len(itinerary) != 0
        flat_route = []
        for sublist in itinerary: #flatten the itinerary
            for port in sublist:
                flat_route.append(port)

        assert len(flat_route) != 0
        travel_distance = sum(overall_distance)
        return flat_route, travel_distance

    def move(self):

        self.step_size = self.ident_distance() #look up the distance between two cities 
        self.state = self.step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, self.next_position) #move the agent
        self.current_route.pop(0) #remove the next step from the itinerary
        self.position = self.next_position
        if len(self.current_route) == 1:
            self.next_position = self.current_route[0] 
        else:
            self.next_position = self.current_route[1] #update current route


    def ident_distance(self): #look up the distance of the current step
        try:
            return self.G.get_edge_data(self.position, self.next_position, default=0)['distance']
        except:
            return 0
    
    def reroute(self):
        self.routes += 1
        self.destination = self.dest()
        
        self.current_route, self.current_dist = self.routing()
        self.state = 3

 
    def step(self):
        self.state = self.state - 1 #'move' ships by one weeks progress
        if self.state <= 0.000: #ships that are en-route to the node they are going to next do not move / perform other activities
            self.distance_traveled += self.step_size #ship has arrived at port, let's add the distance traveled to their 
           
            #add the current position to itinerary
            if self.position != self.current_route[-1]: #if current stop is not the final stop
                new_route, new_distance = self.routing() #perform a new routing to compare against current routing
                
                if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                    # print("default case")
                    self.move()
                    self.itinerary.append(self.position)
        
            # THIS CURRENTLY ONLY CHANGES THE ROUTE IF THE NEXT STEP IS BLOCKED
                elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                    

                    if self.foresight <= (self.current_dist / self.speed): #check how many steps are you from your final destination. If you are far away, do nothing and remain on course
                        if not has_path(self.G, self.position, self.next_position): 
                            self.current_route = new_route
                            self.current_dist = new_distance
                            self.route_chng += 1
                        self.move()
                        self.itinerary.append(self.position)
                    else:
                        self.move()
                        self.itinerary.append(self.position)
                
                
                else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                    # print('reroute: current route longer than new route')
                    self.current_route = new_route
                    self.current_dist = new_distance
                    self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
            
            else: #if ship is arrived at final position, get a new route, and start back
                # print('arrival')
                self.reroute()
                self.itinerary.append(self.position)
        # print("Ship: {}, Source: {}, Destination: {}, Position: {},  Next Stop {}, Time until next Stop {}".format(self.unique_id, self.itinerary[0], self.destination, self.position, self.next_position, self.state ))

    
    # def collect_time

    # def collect_costs

## Model Instances

In [61]:
model = ShippingNetwork(distances, origin, 10)

steps = 600
for i in trange(steps, desc="Stepping Model"):
    model.step()
agent_state = model.datacollector.get_age nt_vars_dataframe()


Placing Ships:  30%|███       | 30/100 [00:04<00:10,  6.87it/s]


NetworkXNoPath: No path to 286.

In [90]:

agent_state


,,Position,Destination,Itinerary,Distance_Traveled,Route,Route Changes
Step,AgentID,,,,,,
1,1,1028.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",9.231781,[24148.0],0
2,1,2000.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",18.463562,[24148.0],0
3,1,6974.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",22.762143,[24148.0],0
4,1,25027.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",30.295111,[24148.0],0
5,1,1975.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",55.884141,[24148.0],0
6,1,6859.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",57.392961,[24148.0],0
7,1,6869.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",119.821445,[24148.0],0
8,1,2052.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",120.871755,[24148.0],0
9,1,2111.0,24148.0,"[2232.0, 1028.0, 2000.0, 6974.0, 25027.0, 1975...",123.940649,[24148.0],0
